# search engine to crawl and scrape pages

In [1]:
import webbrowser                      #immported to open the websites
from tkinter import *                  #to get the front-end
from tkinter import messagebox         # to show the message box
import sqlite3                         #for database
from datetime import date
from tkinter import ttk

#to manipulate the websites using our program we use selenium

from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# to make my computer speak
import win32com.client as wincl


#time is imported to make the request by waiting so that it will not distinguish it is a user or a computer requesting
#import time                     


#CREATING THE TKINTER WINDOWS
w1 = Tk()                             #creating a window 
#w1.configure(background='violet')
str1 = StringVar()                # to store the searched part
str2 = StringVar()                #to store the webpage#str1 is a string variable

w1.title("SEARCH ENGINE")              #giving the window title 
w1.geometry("1200x600+10+10")       #setting the dimension of the window
w1_label = ttk.Label( text ='Website',font=('arial',15,'italic'))
w1_label.place(x=0,y=10)

#CREATING THE DATA BASE AND CREATING THE REQUIRED TABLES
db = sqlite3.connect("D://searched.db")    #SEARCHED.DB
cursor1 =db.cursor()                    #creating the cursor


cursor1.execute("SELECT LINK FROM WEBSITE")
list1 =[]
for row in cursor1.fetchall():
    list1 =list1+[row]        
     
w1_text1 = ttk.Combobox(w1,textvar = str2 ,width = 100)
w1_text1['values']=list1
w1_text1.place(x=80,y=10)
w1_text1.current()  

#cursor1.execute("DROP TABLE WEBSITE")
#cursor1.execute("DROP TABLE HISTORY")
cursor1.execute("DROP TABLE RESULT")
#cursor1.execute("DROP TABLE TEXTSEARCH")

cursor1.execute("CREATE TABLE IF NOT EXISTS 'WEBSITE'(WID INTEGER PRIMARY KEY AUTOINCREMENT,LINK VARCHAR(20) NOT NULL,WNAME VARCHAR(20) NOT NULL)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'HISTORY'(HID INTEGER PRIMARY KEY AUTOINCREMENT,HNAME VARCHAR(20) NOT NULL,HDATE DATE)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'RESULT'(RID INTEGER PRIMARY KEY AUTOINCREMENT,NSEARCH INTEGER NOT NULL,RWEBSITE VARCHAR(20) NOT NULL)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'TEXTSEARCH'(TID INTEGER PRIMARY KEY AUTOINCREMENT,TNAME VARCHAR(20) NOT NULL)")

cursor1.execute("CREATE TABLE IF NOT EXISTS 'WEB_SEARCH'(WID INTEGER,HID INTEGER,CONSTRAINT FK1 FOREIGN KEY (WID) REFERENCES WEBSITE (WID) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT FK2 FOREIGN KEY (HID) REFERENCES HISTORY (HID) ON DELETE CASCADE ON UPDATE CASCADE)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'TEXT_SEARCH'(TID INTEGER,HID INTEGER,CONSTRAINT FK3 FOREIGN KEY (TID) REFERENCES TEXTSEARCH (TID) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT FK4 FOREIGN KEY (HID) REFERENCES HISTORY (HID) ON DELETE CASCADE ON UPDATE CASCADE)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'WEB_RESULT'(WID INTEGER,RID INTEGER,CONSTRAINT FK5 FOREIGN KEY (WID) REFERENCES WEBSITE (WID) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT FK6 FOREIGN KEY (RID) REFERENCES RESULT (RID) ON DELETE CASCADE ON UPDATE CASCADE)")
cursor1.execute("CREATE TABLE IF NOT EXISTS 'T_RESULT'(TID INTEGER,RID INTEGER,CONSTRAINT FK7 FOREIGN KEY (TID) REFERENCES TEXTSEARCH (TID) ON DELETE CASCADE ON UPDATE CASCADE,CONSTRAINT FK8 FOREIGN KEY (RID) REFERENCES RESULT (RID) ON DELETE CASCADE ON UPDATE CASCADE)")

db.commit()


# function defined when we click go
def click():           
    #getting the inputed data
    b = str2.get()
    #storing the data in history table and commiting
    cursor1.execute("INSERT INTO 'HISTORY'(HNAME,HDATE) VALUES(?,?)",(b,date.today()))
    db.commit()
    
    #checcking if the data starts with www or ends with .com,.in,.org,.net
    if(b.startswith('www.') and b.endswith('.com' or '.in' or '.org' or '.net')):
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("you have searched for")
            speak.Speak(b)
        
        
        list1 =list()
        list1 =b.split('.')            #to get the web name
        cursor1.execute("INSERT INTO 'WEBSITE'(LINK,WNAME) VALUES(?,?)",(b,list1[1]))
        db.commit()
        webbrowser.open(b)
        
    #checking if the data ends with com,in,org
    elif(b.endswith('.com' or '.in' or '.org')):
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("you have searched for")
            speak.Speak(b)
        
        
        b ='www.'+b
        cursor1.execute("INSERT INTO 'WEBSITE'(LINK,WNAME) VALUES(?,?)",(b,b.split('.')[1]))
        db.commit()
        webbrowser.open(b)
        
    #checking if the data has blank space in it
    elif(' ' in b):
            w2=Tk()
            w2.title("searched")
            w2.geometry("500x500+0+0")
            speak = wincl.Dispatch("SAPI.SpVoice")
        
            for i in range(1):
                speak.Speak("you have searched for")
                speak.Speak(b)
        
        
            
            x=[]
            x=b.split()
            c='+'.join(x)
            list1=["youtube","google","bing"]
            speak = wincl.Dispatch("SAPI.SpVoice")
        
            for i in range(1):
                speak.Speak("please wait while the web is crawling")
                #speak.Speak(b)
        
            driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")#path where chrome driver is installed
            for lists in list1:
                if(lists=="youtube"):
                    driver.get("https://www."+lists+".com/results?search_query="+c)
                else:
                    driver.get("https://www."+lists+".com/search?q="+c)
                links = driver.find_elements(By.TAG_NAME,"a")
                x_axis=0
                y_axis=0
                def click3(link):
                    webbrowser.open(link)
                count=1    

                for tag in links:
                    url=tag.get_attribute('href')
                    #print(url)
                    if(str(url).startswith("http") or str(url).startswith("https")):
                        button = ttk.Button(w2,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                        y_axis =y_axis+25
                        #importing sleep so that the websites willnot know it is pearson or robot requesting
                        #time.sleep(5)
                        cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                        count=count+1
                        db.commit()
                driver.refresh()
                
            for i in range(1):
                speak.Speak("web crawling is completed")
                
            w2.mainloop()
            #w2.tkraise()
            driver.quit()
        
        
            
    #checking the entry is empty or not
    elif(len(b)!=0):
        w2=Tk()
        w2.title("searched")
        w2.geometry("500x500+0+0")
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("you have searched for")
            speak.Speak(b)
    
        #list of pages to crawl
        list1=["youtube","google","bing"]
        
        driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("please wait while the web is crawling")
            #speak.Speak(b)
        
        for lists in list1:
            if(lists=="youtube"):
                driver.get("https://www."+lists+".com/results?search_query="+b)
            else:
                driver.get("https://www."+lists+".com/search?q="+b)
            links = driver.find_elements(By.TAG_NAME,"a")
            x_axis=0
            y_axis=0
            def click3(link):
                webbrowser.open(link)
            count=1    
            
            for tag in links:
                url=tag.get_attribute('href')
                print(url)
                if(str(url).startswith("http") or str(url).startswith("https")):
                    button = ttk.Button(w2,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                    y_axis =y_axis+25
                    #time.sleep(5)
                    cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                    count=count+1
                    db.commit()
            driver.refresh()
            
        for i in range(1):
            speak.Speak("web crawling is completed")

        w2.mainloop()
        #w2.tkraise()
        driver.quit()
                
    # if there is no input
    else:
        speak1 ="PLEASE ENTER THE TEXT TO SEARCH"
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("error")
            speak.Speak(speak1)
            
        messagebox.showinfo("ERROR",speak1)
        


button = ttk.Button(text ='Go',command = click).place(x = 700,y=10)

#CREATING A LABEL FOR NICE DISPLAY
label1 = Label(text = 'D',fg='dark green',bg="red",font =('arial',15,'italic')).place(x = 550,y =150)
label1 = Label(text = 'E',fg='red',bg="black",font =('arial',15,'italic')).place(x = 570,y =150)
label1 = Label(text = 'E',fg='black',bg="blue",font =('arabic',15,'italic')).place(x =590,y =150)
label1 = Label(text = 'P',fg='blue',bg="white",font =('arial',15,'italic')).place(x = 610,y =150)
label1 = Label(text = 'E',fg='pink',bg="blue",font =('arial',15,'italic')).place(x = 630,y=150)
label1 = Label(text = 'N',fg ='black',bg="red",font =('arial',15,'italic')).place(x = 650,y =150)
label1 = Label(text = 'D',fg ='red',bg="black",font =('arial',15,'italic')).place(x = 670,y =150)

#FUNCTION WHEN WE CLICK SEARCH IN THE TKINTER WINDOW
def click1():
    #getting the user input
    b = str1.get()
    
    #saving the text input in the history
    cursor1.execute("INSERT INTO 'HISTORY'(HNAME,HDATE) VALUES(?,?)",(b,date.today()))
    #cursor1.execute("INSERT INTO 'TEXTSEARCH'(TNAME) VALUES(?)",(b))
    db.commit()
    
    #checking if there is blank space in the data
    if(' ' in b):
        w3=Tk()
        w3.title(b)
        w3.geometry("500x500+0+0")
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("you have searched for")
            speak.Speak(b)
        
        
        x_axis=0
        y_axis=0

        def click3(text):
            webbrowser.open(text)
        count =1
        
        x =[]
        x=b.split()
        c='+'.join(x)
       
        db.commit()
    
        #list of websites to crawl
        list1=["youtube","bing","google"]
        #opening chromedriver
        driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("please wait while the web is crawling")
            #speak.Speak(b)
        

        for lists in list1:
            
            if(lists=="youtube"):
                driver.get("https://www."+lists+".com/results?search_query="+c)
                
            else:
                driver.get("https://www."+lists+".com/search?q="+c)
                
            links = driver.find_elements(By.TAG_NAME,"a")
            x_axis=0
            y_axis=0
            def click3(link):
                webbrowser.open(link)
            count=1    
            
            for tag in links:
                url=tag.get_attribute('href')
                print(url)
                if(str(url).startswith("http") or str(url).startswith("https")):
                    button = ttk.Button(w3,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                    y_axis =y_axis+25
                    #time.sleep(5)
                    cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                    count=count+1
                    db.commit()
        driver.refresh()
        for i in range(1):
            speak.Speak("web crawling is completed")

        w3.mainloop()
        #w3.tkraise()  
        driver.quit()
            

    #checking if there is something to  search
    elif(len(b)!=0):
        w3=Tk()
        w3.title(b)
        
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("you have searched for")
            speak.Speak(b)
        
        w3.geometry("500x500+0+0")
        #list of pages to crawl
        list1=["youtube","google","bing"]
        #cursor1.execute("INSERT INTO 'TEXTSEARCH'(TNAME) VALUES(?)",(b))
        #db.commit()
        
        driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")  #location of selenium webdriver
        
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("please wait while the web is crawling")
            #speak.Speak(b)
        
        for lists in list1:
            if(lists=="youtube"):
                driver.get("https://www."+lists+".com/results?search_query="+b)
            else:
                driver.get("https://www."+lists+".com/search?q="+b)
                
            links = driver.find_elements(By.TAG_NAME,"a")
            x_axis=0
            y_axis=0
            def click3(link):
                webbrowser.open(link)
            count=1    

            for tag in links:
                url=tag.get_attribute('href')
                print(url)
                if(str(url).startswith("http") or str(url).startswith("https")):
                    button = ttk.Button(w3,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                    y_axis =y_axis+25
                    cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                    count=count+1
                    #time.sleep(5)
                    db.commit()
            driver.refresh()

        for i in range(1):
            speak.Speak("web crawling is completed")
            
        w3.mainloop()
        #w3.tkraise()
        driver.quit()    
       
    #if there is nothing inputted
    else:
        speak1 ="PLEASE ENTER THE TEXT TO SEARCH"
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("error")
            speak.Speak(speak1)
            
        messagebox.showinfo("ERROR",speak1)
        



        

# forming a combobox
cursor1.execute("SELECT HNAME FROM HISTORY")
list1 =[]
for row in cursor1.fetchall():
    list1 =list1+[row]        
     
w1_text2 = ttk.Combobox(w1,textvar = str1 ,width = 50)
w1_text2['values']=list1
w1_text2.place(x=500,y=200)
w1_text2.current()  
button1 = ttk.Button(text = 'Search',command = click1).place(x = 600,y =250)

#FUNCTION TO KILL THE WINDOW
def quit1():
    mess = messagebox.askyesno(title = 'Quit', message ='Are you sure you want to quit?')   #function to ask the user
    if(mess==1):
        w1.destroy()           

#CREATING THE MENU
list1 = Menu()
list1.add_command(label='quit',command = quit1)

def new1():
    w4 = Tk()
    w4.title("SEARCH ENGINE")
    w4.geometry("1000x1000+0+0")
    
list2 =Menu()
list2.add_command(label='New Tab',command =new1)

def setting():
    w5 =Tk()
    w5.title("HISTORY")
    w5.geometry("500x500+100+100")
    
    def Show():
    
        scrollbar = Scrollbar(w5)
        scrollbar.pack( side = RIGHT, fill=Y )
    
        cursor1.execute("SELECT * FROM HISTORY")
        mylist = Listbox(w5, yscrollcommand = scrollbar.set ,width=80)
        
        for row in cursor1.fetchall():
            mylist.insert(END,str(row))
            
        mylist.pack( side = LEFT, fill = BOTH )
        scrollbar.config( command = mylist.yview )
        speak = wincl.Dispatch("SAPI.SpVoice")

        for i in range(1):
            speak.Speak("The history that are stored in databases are")
     
        w5.mainloop()
        

    but1 =Button(w5,text="Show History",font=('aerial',10),command=Show).place(x=1,y=0)
    def Clear():
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        text1="ARE YOU SURE YOU WANT TO DELETE HISTORY?"

        for i in range(1):
            speak.Speak(text1)

        temp = messagebox.askyesno("CONFIRMATION","ARE YOU SURE YOU WANT TO DELETE HISTORY?")
        if(temp==1):
            cursor1.execute("DELETE FROM 'HISTORY'")
            db.commit()
            
            text2 = "HISTORY HAS BEEN DELETED"
            speak.Speak(text2)
            label1 = ttk.Label(w5,text=text2,font=('aerial',20))
            label1.place(x=100,y=100)
    
    but2 =Button(w5,text="Clear History",font=('aerial',10),command=Clear).place(x=2,y=40)
    w5.mainloop()
    
def func1():
    w5 =Tk()
    w5.title("SEARCHED SITES")
    w5.geometry("500x500+100+100")
    xvalue=1
    yvalue=30
    
    Label(w5,text='SEARCHED SITES',font=('aerial',20)).pack()
    cursor1.execute("SELECT * FROM WEBSITE")
    
    for row in cursor1.fetchall():
        label1 =ttk.Label(w5,text=row)
        label1.place(x=xvalue,y=yvalue)
        yvalue=yvalue+20
    
    speak = wincl.Dispatch("SAPI.SpVoice")

    for i in range(1):
        speak.Speak("The searched web sites are")
        #speak.Speak("search any thing you want to search")

    w5.mainloop()
list3 = Menu()
list3.add_command(label='History',command = setting)
list3.add_command(label='Searched Sites',command =func1)

menu1 = Menu()                           #calling menu inbuilt function
menu1.add_cascade(label = 'New Tab',font=20)       
menu1.add_cascade(label = 'X',font=20,menu = list1)           #adding menu
menu1.add_cascade(label = '+',font=20,menu = list2)           #adding menu
menu1.add_cascade(label = 'Setting',font = 20,menu = list3)

w1.config(menu=menu1)       #to configure the menu items


#LOGIC TO GET THE MOSTLY VISITED SITES AND SEARHCED THINGS

cursor1.execute("SELECT HNAME,COUNT(*) FROM HISTORY WHERE HNAME !='' GROUP BY HNAME ORDER BY COUNT(*) DESC")
count =0    
x_value = 0
y_value = 580

#if we click the PREDICTED BUTTON MADED FROM HISTORY
def on_click(b):
    #print(b)
    speak = wincl.Dispatch("SAPI.SpVoice")
        
    for i in range(1):
        speak.Speak("you have clicked")
        speak.Speak(b)

        
    if(b.startswith('www.') and b.endswith('.com' or '.in' or '.org' or '.net')):
        webbrowser.open(b)
        
    #checking if the data ends with com,in,org
    elif(b.endswith('.com' or '.in' or '.org')):
        b ='www.'+b
        webbrowser.open(b)
        
    #checking if the data has blank space in it
    elif(' ' in b):
            w2=Tk()
            w2.title("searched")
            w2.geometry("500x500+0+0")
            x=[]
            x=b.split()
            c='+'.join(x)
            
            list1=["youtube","google","bing"]
            
            speak = wincl.Dispatch("SAPI.SpVoice")
        
            for i in range(1):
                speak.Speak("please wait while the web is crawling")
                #speak.Speak(b)
        
            driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")#path where chrome driver is installed
            
            for lists in list1:
                if(lists=="youtube"):
                    driver.get("https://www."+lists+".com/results?search_query="+c)
                else:
                    driver.get("https://www."+lists+".com/search?q="+c)
                links = driver.find_elements(By.TAG_NAME,"a")
                x_axis=0
                y_axis=0
                def click3(link):
                    webbrowser.open(link)
                count=1    

                for tag in links:
                    url=tag.get_attribute('href')
                    #print(url)
                    if(str(url).startswith("http") or str(url).startswith("https")):
                        button = ttk.Button(w2,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                        y_axis =y_axis+25
                        #importing sleep so that the websites willnot know it is pearson or robot requesting
                        #time.sleep(5)
                        cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                        count=count+1
                        db.commit()
                driver.refresh()
                
            for i in range(1):
                speak.Speak("web crawling is completed")
                
            w2.mainloop()
            #w2.tkraise()
            driver.quit()
        
        
            
    #checking the entry is empty or not
    elif(len(b)!=0):
        w2=Tk()
        w2.title("searched")
        w2.geometry("500x500+0+0")
        
        #list of pages to crawl
        list1=["youtube","google","bing"]
        
        driver = webdriver.Chrome(r"C:\Users\Anon\Desktop\chromedriver.exe")
        speak = wincl.Dispatch("SAPI.SpVoice")
        
        for i in range(1):
            speak.Speak("please wait while the web is crawling")
            #speak.Speak(b)
        
        for lists in list1:
            if(lists=="youtube"):
                driver.get("https://www."+lists+".com/results?search_query="+b)
            else:
                driver.get("https://www."+lists+".com/search?q="+b)
            links = driver.find_elements(By.TAG_NAME,"a")
            x_axis=0
            y_axis=0
            def click3(link):
                webbrowser.open(link)
            count=1    
            
            for tag in links:
                url=tag.get_attribute('href')
                print(url)
                if(str(url).startswith("http") or str(url).startswith("https")):
                    button = ttk.Button(w2,text=url,command= lambda s=url:click3(s)).place(x=x_axis,y=y_axis)
                    y_axis =y_axis+25
                    #time.sleep(5)
                    cursor1.execute("INSERT INTO 'RESULT'(NSEARCH,RWEBSITE) VALUES(?,?)",(count,url.split('.')[0]))
                    count=count+1
                    db.commit()
            driver.refresh()
            
        for i in range(1):
            speak.Speak("web crawling is completed")

        w2.mainloop()
        #w2.tkraise()
        driver.quit()
    
    
    
for row in cursor1.fetchall():
    #print(row[1])
    if(count!=10):
        #row[0] has the HISTORY_NAME IN IT
        pedict_button = ttk.Button(w1,text = row[0],command= lambda s=row[0]:on_click(s)).place(x = x_value,y =y_value,anchor=SW)
        x_value = x_value+100
        count =count+1
    else:
        break

    
#code to make my computer speak
speak = wincl.Dispatch("SAPI.SpVoice")

for i in range(1):
    speak.Speak("welcome to search engine made by deepen shrestha")
    speak.Speak("you can search any thing you want")


w1.mainloop()                  # TO DISPLAY THE TKINTER WINDOW
speak.Speak("thanks for visiting us. please do visit us more.")
    

1